### PoC Nutrition asked queries :

**Table of content :** <br>
[1️⃣ | Nombre de produits par nutriscore. Quel est le pourcentage de représentation associé?](#1) <br>
[2️⃣ | Les 4 valeurs possibles du Nova_Group](#2) <br>
[3️⃣ | Quel est la moyenne des fibres par nutriscore ?](#3) <br>
[4️⃣ | Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs](#4) <br>
[5️⃣ | Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B](#5) <br>
[6️⃣ | Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres](#6) <br>

In [1]:
import psycopg2 as pgconn
import pandas as pd

print('⬆️ This contains imports & pg connector.')

# Init db connection and open cursor
def db_connect(cred):
    return pgconn.connect(
                host=cred['host'],
                database=cred['database'],
                user=cred['user'],
                password=cred['password']
            )

credentials = {
            'host': 'localhost',
            'database': 'open_food_data_db',
            'user': 'postgres',
            'password': ''
        }

conn = db_connect(credentials)
cur = conn.cursor()

⬆️ This contains imports & pg connector.


> ***Nota Bene*** <br>
> Les résultats obtenus ne sont pas sur l'ensemble des données. <br>
> Les queries ont été rédigées dans une syntaxe compatible à postgres.

-----
#### 1️⃣ | Nombre de produits par nutriscore. Quel est le pourcentage de représentation associé? <a name=1></a>

```sql
SELECT
  nutriscore_score,
  cnt,
  ROUND(100.0 * (( cnt) / (sum(cnt) OVER ())), 2) AS percentage
FROM (
	SELECT nutriscore_score, 
	count(*) AS cnt
	FROM records
	WHERE nutriscore_score::FLOAT IS NOT NULL 
	GROUP BY nutriscore_score
	ORDER BY cnt DESC
) x;
```

In [2]:
cur.execute('SELECT nutriscore_score, cnt, ROUND(100.0 * (( cnt) / (sum(cnt) OVER ())), 2) AS percentage FROM ( SELECT nutriscore_score,  count(*) AS cnt FROM records WHERE nutriscore_score::FLOAT IS NOT NULL  GROUP BY nutriscore_score ORDER BY cnt DESC ) x;')
res = cur.fetchall()

print('\nNombre de produits par nutriscore et % associés : \n')
df = pd.DataFrame(res, columns=['nutriscore_score', 'n count', '%'])
print(df.to_string(index=False))


Nombre de produits par nutriscore et % associés : 

nutriscore_score  n count    %
            14.0    14001 5.49
             0.0    11793 4.63
            13.0    11360 4.46
            16.0    11312 4.44
            15.0    10997 4.31
            11.0    10963 4.30
            12.0    10651 4.18
             2.0     9585 3.76
             1.0     9145 3.59
            -1.0     9070 3.56
            -4.0     8982 3.52
             3.0     8592 3.37
            10.0     8148 3.20
            17.0     7890 3.10
            -2.0     7556 2.96
            19.0     7382 2.90
             4.0     7352 2.88
            18.0     7136 2.80
            -3.0     6742 2.65
            20.0     6630 2.60
             5.0     6461 2.53
             9.0     5889 2.31
             6.0     5302 2.08
            21.0     5264 2.07
             8.0     5233 2.05
            24.0     4895 1.92
            23.0     4733 1.86
            -5.0     4659 1.83
             7.0     4642 1.82
            22.0 

-------
#### 2️⃣ | Les 4 valeurs possibles du Nova_Group <a name=2></a>

```sql
SELECT DISTINCT(nova_group) FROM records ORDER BY nova_group DESC;
```

In [3]:
cur.execute('SELECT DISTINCT(nova_group) FROM records ORDER BY nova_group DESC;')
res = cur.fetchall()

print('\nLes 4 valeurs possibles du `nova_group` :\n')
df = pd.DataFrame(res, columns=['Nova group values'])
print(df.to_string(index=False))


Les 4 valeurs possibles du `nova_group` :

Nova group values
             None
              4.0
              3.0
              2.0
              1.0


---------
#### 3️⃣ | Quel est la moyenne des fibres par nutriscore ? <a name=3></a>

```sql
SELECT
	nutriscore_score,
	AVG(fiber_100g::FLOAT) as avg_fiber_per_100g
FROM records
WHERE nutriscore_score::FLOAT IS NOT NULL 
GROUP BY nutriscore_score
ORDER BY avg_fiber_per_100g DESC
```

In [14]:
cur.execute('SELECT nutriscore_score, AVG(fiber_100g::FLOAT) as avg_fiber_per_100g FROM records WHERE nutriscore_score::FLOAT IS NOT NULL GROUP BY nutriscore_score ORDER BY avg_fiber_per_100g DESC')
res = cur.fetchall()

print('\nMoyenne des fibres par nutriscore :\n')
df = pd.DataFrame(res, columns=['Nutriscore', 'Fibre pour 100g'])
df = df.sort_values(by='Nutriscore')
print(df.to_string(index=False))


Moyenne des fibres par nutriscore :

Nutriscore  Fibre pour 100g
      -1.0         3.242917
     -10.0         5.130479
     -11.0         6.642538
     -12.0         6.188640
     -13.0         6.902083
     -14.0         6.976364
     -15.0        20.000000
      -2.0         3.284266
      -3.0         2.908948
      -4.0         3.614725
      -5.0         5.560119
      -6.0         7.127910
      -7.0         5.872383
      -8.0         8.836483
      -9.0         4.663594
       0.0         2.562328
       1.0         1.802210
      10.0         3.906644
      11.0         3.093220
      12.0         2.858084
      13.0         2.481809
      14.0         1.781905
      15.0         1.893567
      16.0         1.732417
      17.0         2.210963
      18.0         2.205383
      19.0         2.092816
       2.0         1.669167
      20.0         2.280234
      21.0         2.385261
      22.0         1.925602
      23.0         1.781046
      24.0         1.569632
      25.0

-----
#### 4️⃣ | Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs <a name='4'></a>

```sql
SELECT
	pnns_groups_2,
	AVG(additives_n::FLOAT) as additives_n
FROM records
GROUP BY pnns_groups_2
ORDER BY additives_n DESC
LIMIT 10;
```

In [16]:
cur.execute('SELECT pnns_groups_2, AVG(additives_n::FLOAT) as additives_n FROM records GROUP BY pnns_groups_2 ORDER BY additives_n DESC LIMIT 10;')
res = cur.fetchall()

print('\nQuels sont le TOP10 des pnns_groups_2 qui comportent le plus d\'additifs\n')
df = pd.DataFrame(res, columns=['pnns_group_2', 'N additifs'])
print(df.to_string(index=False))


Quels sont le TOP10 des pnns_groups_2 qui comportent le plus d'additifs

                    pnns_group_2  N additifs
                      Sandwiches    7.430631
                       Ice cream    6.197309
              Biscuits and cakes    5.942924
          Pizza pies and quiches    5.348089
Artificially sweetened beverages    5.049776
                           Bread    3.821809
                          Sweets    3.817673
                  Dairy desserts    3.066038
                        Pastries    2.811321
    Plant-based milk substitutes    2.748954


------
#### 5️⃣ | Afficher le taux de sucre/fat moyen et médian des produits avec un nutriscore A ou B <a name=5></a>

```sql
SELECT
    nutriscore_grade,
	AVG(sugars_100g::FLOAT) avg_sugar,
	AVG(fat_100g::FLOAT) avg_fat,
	(SELECT percentile_disc(0.5) WITHIN GROUP (order by records.sugars_100g) FROM records) as med_sugar,
	(SELECT percentile_disc(0.5) WITHIN GROUP (order by records.fat_100g) FROM records) as med_fat
FROM records
WHERE nutriscore_grade = 'a' OR nutriscore_grade = 'b'
GROUP BY nutriscore_grade
```

In [17]:
cur.execute('SELECT nutriscore_grade, AVG(sugars_100g::FLOAT) avg_sugar, AVG(fat_100g::FLOAT) avg_fat, (SELECT percentile_disc(0.5) WITHIN GROUP (order by records.sugars_100g) FROM records) as med_sugar, (SELECT percentile_disc(0.5) WITHIN GROUP (order by records.fat_100g) FROM records) as med_fat FROM records WHERE nutriscore_grade = \'a\' OR nutriscore_grade = \'b\' GROUP BY nutriscore_grade')
res = cur.fetchall()

print('\nTaux moyen/medin de sucre/fat pour les produits avec un nutriscore A ou B\n')
df = pd.DataFrame(res, columns=['nutriscore', 'avg surgar', 'avg fat', 'med sugar', 'med fat'])
print(df.to_string(index=False))


Taux moyen/medin de sucre/fat pour les produits avec un nutriscore A ou B

nutriscore  avg surgar  avg fat med sugar med fat
         a    3.811035 4.109291      20.0     2.0
         b    4.914000 5.070053      20.0     2.0


----
#### 6️⃣ | Combien avons-nous de produits qui ont un taux de carbohydrates > aux sucres <a name=6></a>

```sql
SELECT
	COUNT(DISTINCT(code))
FROM records
WHERE carbohydrates_100g::FLOAT > sugars_100g::FLOAT
````

In [18]:
cur.execute('SELECT COUNT(DISTINCT(code)) FROM records WHERE carbohydrates_100g::FLOAT > sugars_100g::FLOAT')
res = cur.fetchall()

print(f'\nNombre de produits ou le taux de carbohydrates > sucres : {res[0][0]}\n')


Nombre de produits ou le taux de carbohydrates > sucres : 308294

